<a href="https://colab.research.google.com/github/shives-tkt/FINAL-PRO3/blob/main/PRO_3_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.tsa.arima.model import ARIMA
import dash
from dash import dcc, html

data = pd.read_csv("path_to_your_dataset.csv")
data.isnull().sum()
data = data.fillna(method='ffill')

data['hour'] = pd.to_datetime(data['timestamp']).dt.hour
data['day_of_week'] = pd.to_datetime(data['timestamp']).dt.dayofweek
data['rain'] = data['weather_condition'].apply(lambda x: 1 if x == 'rain' else 0)

scaler = StandardScaler()
data[['distance', 'temperature']] = scaler.fit_transform(data[['distance', 'temperature']])

data = pd.get_dummies(data, columns=['pickup_location', 'event_type'])

sns.boxplot(x='weather_condition', y='demand', data=data)
plt.show()

sns.scatterplot(x='price', y='demand', data=data)
plt.show()

X = data.drop('price', axis=1)
y = data['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R²: {r2}")

model_arima = ARIMA(y_train, order=(1,1,1))
model_arima_fit = model_arima.fit()
forecast = model_arima_fit.forecast(steps=10)

def dynamic_pricing(current_demand, weather_condition, available_drivers):
    base_price = 10
    demand_factor = 1.5 if current_demand > 50 else 1
    weather_factor = 1.2 if weather_condition == 'rain' else 1
    driver_factor = 0.9 if available_drivers > 10 else 1.1
    return base_price * demand_factor * weather_factor * driver_factor

print(dynamic_pricing(60, 'rain', 5))

sns.scatterplot(x=y_test, y=y_pred)
plt.xlabel('True Prices')
plt.ylabel('Predicted Prices')
plt.title('True vs Predicted Prices')
plt.show()

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Dynamic Pricing Dashboard"),
    dcc.Graph(
        figure={
            'data': [
                {'x': data['hour'], 'y': data['price'], 'type': 'line', 'name': 'Price'},
                {'x': data['hour'], 'y': data['demand'], 'type': 'line', 'name': 'Demand'}
            ],
            'layout': {
                'title': 'Price and Demand over Time'
            }
        }
    ),
    dcc.Graph(
        figure={
            'data': [
                {'x': data['temperature'], 'y': data['price'], 'type': 'scatter', 'mode': 'markers', 'name': 'Price vs Temperature'}
            ],
            'layout': {
                'title': 'Price vs Temperature'
            }
        }
    )
])

app.run_server(debug=True)
